In [1]:
from gc_utils import *

alg = Algebra(5)
ga2 = Algebra(2)
locals().update(alg.blades)

In [2]:
# 2d general linear transform
def random_linear_trans(alg: Algebra):
    matrix = np.random.rand(alg.d, alg.d)
    return matrix2trans(matrix, alg)

def random_ortho_trans(alg: Algebra):
    from scipy.stats import ortho_group
    matrix = ortho_group.rvs(dim=alg.d)
    return matrix2trans(matrix, alg)

f = random_linear_trans(ga2)
x = random_r_blade(1, ga2)
f(x)

0.25 𝐞₁ + 0.467 𝐞₂

In [3]:
# 2d polar decomposition
# f = SUU
df = derivative(f, x, ga2, grade=1)
UU = df / norm(df)
S = lambda x: UU * f(x)
S(x)*UU, f(x)

(0.25 𝐞₁ + 0.467 𝐞₂, 0.25 𝐞₁ + 0.467 𝐞₂)

In [4]:
# UU is rotor, S is symmetric
UU, curl(S, x, ga2, grade=1)

(0.957 + 0.289 𝐞₁₂, -2.78e-11 𝐞₁₂)

In [5]:
#6.3 remove the condition of 2 dimension
a, b, x = random_r_vectors(3, alg)
g = lambda x: x + (x|a)*b
abframe = [a,b]
r_frame = reciprocal(abframe)
dg = 2 + a*b
derivative(g, x, alg, grade=1, frame=abframe, r_frame=r_frame), dg

(3.24 + 0.0124 𝐞₁₂ + -0.224 𝐞₁₃ + -0.13 𝐞₁₄ + -0.3 𝐞₁₅ + -0.17 𝐞₂₃ + -0.0998 𝐞₂₄ + -0.222 𝐞₂₅ + 0.0178 𝐞₃₄ + -0.107 𝐞₃₅ + -0.0857 𝐞₄₅ + 4.82e-17 𝐞₁₂₃₄ + 3.82e-16 𝐞₁₂₃₅ + 9.9e-17 𝐞₁₂₄₅ + -1.17e-16 𝐞₁₃₄₅ + -7.75e-18 𝐞₂₃₄₅,
 3.24 + 0.0124 𝐞₁₂ + -0.224 𝐞₁₃ + -0.13 𝐞₁₄ + -0.3 𝐞₁₅ + -0.17 𝐞₂₃ + -0.0998 𝐞₂₄ + -0.222 𝐞₂₅ + 0.0178 𝐞₃₄ + -0.107 𝐞₃₅ + -0.0857 𝐞₄₅)

In [6]:
#6.3
dg_norm = norm(dg)
UU = dg / dg_norm
U = simple_rotor_sqrt(UU)
S = lambda x: U.sw(g(x))
curl(S, x, alg, grade=1), U.reverse().sw(S(x)), g(x)

(-1.11e-10 𝐞₁₂ + -5.55e-10 𝐞₁₃ + 3.33e-10 𝐞₁₅ + 6.66e-10 𝐞₂₃ + -5.55e-10 𝐞₂₄ + -5.55e-10 𝐞₂₅ + 5.55e-11 𝐞₃₄ + 4.44e-10 𝐞₃₅ + -2.22e-10 𝐞₄₅,
 1.66 𝐞₁ + 1.55 𝐞₂ + 0.996 𝐞₃ + 0.558 𝐞₄ + 1.04 𝐞₅ + -8.67e-19 𝐞₁₂₃₄₅,
 1.66 𝐞₁ + 1.55 𝐞₂ + 0.996 𝐞₃ + 0.558 𝐞₄ + 1.04 𝐞₅)

In [7]:
#6.6
frame = alg.frame
i, j = 1, 3
aij = (1/np.sqrt(2))*(frame[i] + frame[j])
[aij.sw(x) for x in frame]

[-1.0 𝐞₁, 1.0 𝐞₄, -1.0 𝐞₃, 1.0 𝐞₂, -1.0 𝐞₅]

In [8]:
#6.7
alpha = 3
Sij = lambda x: x + (alpha * (x|frame[i]) * frame[j])
[Sij(x) for x in frame]

[1 𝐞₁, 1 𝐞₂ + 3 𝐞₄, 1 𝐞₃, 1 𝐞₄, 1 𝐞₅]

In [9]:
#7.2
g = random_linear_trans(alg)
g_ = lambda x, y: x | g(y)
derivative(lambda x: g_(x, b), x, alg, grade=1), g(b)

(2.08 𝐞₁ + 1.15 𝐞₂ + 1.03 𝐞₃ + 1.13 𝐞₄ + 0.951 𝐞₅,
 2.08 𝐞₁ + 1.15 𝐞₂ + 1.03 𝐞₃ + 1.13 𝐞₄ + 0.951 𝐞₅)

In [10]:
# check nonsingular
outermorphism(g, alg.pseudoscalar((1,)), alg)

-0.0706 𝐞₁₂₃₄₅

In [11]:
#7.5 orthogonal transformation is its isometry
f = random_ortho_trans(alg)
x | f(b), f(x) | f(f(b))

(-0.914, -0.914)

In [12]:
# skew bilinear form
gm = skew_part(g, alg)
G = simplicial_derivative(lambda vectors: vectors[0] | gm(vectors[1]), np.zeros(2), alg)
x | gm(b), x | (b | G)

(-0.205, -0.205)

In [13]:
#7.6b f is not an isometry of gm
adjoint_outermorphism(f, G, alg), G

(0.355 𝐞₁₂ + -0.217 𝐞₁₃ + 0.00296 𝐞₁₄ + -0.223 𝐞₁₅ + 0.145 𝐞₂₃ + 0.0784 𝐞₂₄ + -0.194 𝐞₂₅ + 0.0769 𝐞₃₄ + -0.0521 𝐞₃₅ + 0.138 𝐞₄₅,
 -0.349 𝐞₁₂ + 0.086 𝐞₁₃ + 0.118 𝐞₁₄ + -0.156 𝐞₁₅ + -0.129 𝐞₂₃ + -0.291 𝐞₂₄ + -0.0631 𝐞₂₅ + 0.137 𝐞₃₄ + -0.156 𝐞₃₅ + -0.017 𝐞₄₅)

In [14]:
#7.8 the inverse of gm
blades = bivector_split(G, alg)
gm(sum(-x | blade/normsq(blade) for blade in blades)), x

(0.428 𝐞₁ + 0.776 𝐞₂ + 0.567 𝐞₃ + -0.119 𝐞₄ + 0.628 𝐞₅,
 0.584 𝐞₁ + 0.763 𝐞₂ + 0.39 𝐞₃ + 0.121 𝐞₄ + 0.742 𝐞₅)

In [15]:
#7.12 assume splitted blades are normal
blades_norm = [norm(b) for b in blades]
blades = [b / n for b, n in zip(blades, blades_norm)]
gprod(blades)

-3.75e-16 + 2.78e-17 𝐞₁₂ + -2.78e-17 𝐞₁₄ + -1.39e-17 𝐞₁₅ + -2.43e-17 𝐞₂₃ + -2.08e-17 𝐞₂₄ + -8.33e-17 𝐞₂₅ + 2.78e-17 𝐞₃₄ + 6.94e-17 𝐞₃₅ + 4.16e-17 𝐞₄₅ + -0.321 𝐞₁₂₃₄ + 0.677 𝐞₁₂₃₅ + 0.496 𝐞₁₂₄₅ + -0.0353 𝐞₁₃₄₅ + -0.439 𝐞₂₃₄₅

In [16]:
#7.17
theta = np.random.rand(2)
rotors = [blade_exp(t*blade) for t, blade in zip(theta, blades)]
[(U.sw(G), G) for U in rotors for G in blades]

[(-0.00213 𝐞₁₂ + -0.217 𝐞₁₃ + 0.0336 𝐞₁₄ + -0.407 𝐞₁₅ + -0.19 𝐞₂₃ + 0.0315 𝐞₂₄ + -0.363 𝐞₂₅ + 0.204 𝐞₃₄ + -0.707 𝐞₃₅ + -0.274 𝐞₄₅,
  -0.00213 𝐞₁₂ + -0.217 𝐞₁₃ + -0.19 𝐞₂₃ + 0.0336 𝐞₁₄ + 0.0315 𝐞₂₄ + 0.204 𝐞₃₄ + -0.407 𝐞₁₅ + -0.363 𝐞₂₅ + -0.707 𝐞₃₅ + -0.274 𝐞₄₅),
 (-0.68 𝐞₁₂ + 0.266 𝐞₁₃ + 0.216 𝐞₁₄ + -0.12 𝐞₁₅ + -0.166 𝐞₂₃ + -0.581 𝐞₂₄ + 0.0406 𝐞₂₅ + 0.174 𝐞₃₄ + 0.0135 𝐞₃₅ + 0.09 𝐞₄₅,
  -0.68 𝐞₁₂ + 0.266 𝐞₁₃ + -0.166 𝐞₂₃ + 0.216 𝐞₁₄ + -0.581 𝐞₂₄ + 0.174 𝐞₃₄ + -0.12 𝐞₁₅ + 0.0406 𝐞₂₅ + 0.0135 𝐞₃₅ + 0.09 𝐞₄₅),
 (-0.00213 𝐞₁₂ + -0.217 𝐞₁₃ + 0.0336 𝐞₁₄ + -0.407 𝐞₁₅ + -0.19 𝐞₂₃ + 0.0315 𝐞₂₄ + -0.363 𝐞₂₅ + 0.204 𝐞₃₄ + -0.707 𝐞₃₅ + -0.274 𝐞₄₅,
  -0.00213 𝐞₁₂ + -0.217 𝐞₁₃ + -0.19 𝐞₂₃ + 0.0336 𝐞₁₄ + 0.0315 𝐞₂₄ + 0.204 𝐞₃₄ + -0.407 𝐞₁₅ + -0.363 𝐞₂₅ + -0.707 𝐞₃₅ + -0.274 𝐞₄₅),
 (-0.68 𝐞₁₂ + 0.266 𝐞₁₃ + 0.216 𝐞₁₄ + -0.12 𝐞₁₅ + -0.166 𝐞₂₃ + -0.581 𝐞₂₄ + 0.0406 𝐞₂₅ + 0.174 𝐞₃₄ + 0.0135 𝐞₃₅ + 0.09 𝐞₄₅,
  -0.68 𝐞₁₂ + 0.266 𝐞₁₃ + -0.166 𝐞₂₃ + 0.216 𝐞₁₄ + -0.581 𝐞₂₄ + 0.174 𝐞₃₄ + -0.12 𝐞₁₅ + 0.0406 𝐞₂₅ + 

In [17]:
#7.18
first_vecs = [blade_split(blade, alg)[0] for blade in blades]
first_vecs = [v/norm(v) for v in first_vecs]
second_vecs = [v | blade for v, blade in zip(first_vecs, blades)]
inter_plane_rotors = [simple_rotor_sqrt(a * b) for a, b in (first_vecs, second_vecs)]
U = gprod(inter_plane_rotors)
terms_ratio(U.sw(blades[0]), blades[1])

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [18]:
f"{x[1:]}"

'(0.7632779671663076, 0.3904474196857215, 0.12121583763530885, 0.7421779382505789)'

In [19]:
G_normalized = sum(blades)
G_normalized, U.sw(G_normalized)

(-0.682 𝐞₁₂ + 0.0487 𝐞₁₃ + 0.25 𝐞₁₄ + -0.527 𝐞₁₅ + -0.356 𝐞₂₃ + -0.55 𝐞₂₄ + -0.323 𝐞₂₅ + 0.379 𝐞₃₄ + -0.694 𝐞₃₅ + -0.184 𝐞₄₅,
 -0.682 𝐞₁₂ + 0.0487 𝐞₁₃ + 0.25 𝐞₁₄ + -0.527 𝐞₁₅ + -0.356 𝐞₂₃ + -0.55 𝐞₂₄ + -0.323 𝐞₂₅ + 0.379 𝐞₃₄ + -0.694 𝐞₃₅ + -0.184 𝐞₄₅)

In [20]:
# we can recover this "normalization"
sum(P(G_normalized, b) * n for b, n in zip(blades, blades_norm)), G

(-0.349 𝐞₁₂ + 0.086 𝐞₁₃ + 0.118 𝐞₁₄ + -0.156 𝐞₁₅ + -0.129 𝐞₂₃ + -0.291 𝐞₂₄ + -0.0631 𝐞₂₅ + 0.137 𝐞₃₄ + -0.156 𝐞₃₅ + -0.017 𝐞₄₅,
 -0.349 𝐞₁₂ + 0.086 𝐞₁₃ + 0.118 𝐞₁₄ + -0.156 𝐞₁₅ + -0.129 𝐞₂₃ + -0.291 𝐞₂₄ + -0.0631 𝐞₂₅ + 0.137 𝐞₃₄ + -0.156 𝐞₃₅ + -0.017 𝐞₄₅)

In [21]:
#7.21 zero if not onto the decomposed blades 
sympletic_basis = first_vecs + second_vecs
list(zip((v | (w | G_normalized) for v, w in product(sympletic_basis, repeat=2)), (product(['f', 'fp', 's', 'sp'], repeat=2))))

[(-2.78e-17, ('f', 'f')),
 (4.16e-17, ('f', 'fp')),
 (-1.0, ('f', 's')),
 (9.11e-17, ('f', 'sp')),
 (-1.08e-16, ('fp', 'f')),
 (1.89e-17, ('fp', 'fp')),
 (2.37e-16, ('fp', 's')),
 (-1.0, ('fp', 'sp')),
 (1.0, ('s', 'f')),
 (-1.67e-16, ('s', 'fp')),
 (, ('s', 's')),
 (-7.29e-17, ('s', 'sp')),
 (-1.11e-16, ('sp', 'f')),
 (1.0, ('sp', 'fp')),
 (7.63e-17, ('sp', 's')),
 (-5.79e-17, ('sp', 'sp'))]

In [22]:
# Recover the normalization
[gm(v)/n for v, n in zip(sympletic_basis, blades_norm*2)], sympletic_basis

([-0.274 𝐞₁ + -0.241 𝐞₂ + -2.41e-16 𝐞₃ + -0.258 𝐞₄ + 0.894 𝐞₅,
  -1.08e-16 𝐞₁ + 0.882 𝐞₂ + -0.345 𝐞₃ + -0.281 𝐞₄ + 0.156 𝐞₅,
  0.372 𝐞₁ + 0.334 𝐞₂ + 0.791 𝐞₃ + 0.228 𝐞₄ + 0.27 𝐞₅,
  0.771 𝐞₁ + -0.227 𝐞₂ + -0.0996 𝐞₃ + -0.587 𝐞₄ + 0.00585 𝐞₅],
 [-0.372 𝐞₁ + -0.334 𝐞₂ + -0.791 𝐞₃ + -0.228 𝐞₄ + -0.27 𝐞₅,
  -0.771 𝐞₁ + 0.227 𝐞₂ + 0.0996 𝐞₃ + 0.587 𝐞₄ + -0.00585 𝐞₅,
  -0.274 𝐞₁ + -0.241 𝐞₂ + 2.78e-17 𝐞₃ + -0.258 𝐞₄ + 0.894 𝐞₅,
  1.65e-17 𝐞₁ + 0.882 𝐞₂ + -0.345 𝐞₃ + -0.281 𝐞₄ + 0.156 𝐞₅])

In [23]:
#7.26a symplectic transformations of gm (isometries of skew bilinear form)
# conditions: unity and sympletic basis
translation = lambda x: (
    x + 
    2*(x|second_vecs[1]) * first_vecs[1] + 
    3*(x|second_vecs[0]) * first_vecs[0]
    )

outermorphism(translation, G_normalized, alg), G_normalized, outermorphism(translation, G, alg), G

(-0.682 𝐞₁₂ + 0.0487 𝐞₁₃ + 0.25 𝐞₁₄ + -0.527 𝐞₁₅ + -0.356 𝐞₂₃ + -0.55 𝐞₂₄ + -0.323 𝐞₂₅ + 0.379 𝐞₃₄ + -0.694 𝐞₃₅ + -0.184 𝐞₄₅,
 -0.682 𝐞₁₂ + 0.0487 𝐞₁₃ + 0.25 𝐞₁₄ + -0.527 𝐞₁₅ + -0.356 𝐞₂₃ + -0.55 𝐞₂₄ + -0.323 𝐞₂₅ + 0.379 𝐞₃₄ + -0.694 𝐞₃₅ + -0.184 𝐞₄₅,
 -0.349 𝐞₁₂ + 0.086 𝐞₁₃ + 0.118 𝐞₁₄ + -0.156 𝐞₁₅ + -0.129 𝐞₂₃ + -0.291 𝐞₂₄ + -0.0631 𝐞₂₅ + 0.137 𝐞₃₄ + -0.156 𝐞₃₅ + -0.017 𝐞₄₅,
 -0.349 𝐞₁₂ + 0.086 𝐞₁₃ + 0.118 𝐞₁₄ + -0.156 𝐞₁₅ + -0.129 𝐞₂₃ + -0.291 𝐞₂₄ + -0.0631 𝐞₂₅ + 0.137 𝐞₃₄ + -0.156 𝐞₃₅ + -0.017 𝐞₄₅)

In [24]:
#7.26b
rotation = lambda x: (
    x + 
    4*((x|first_vecs[1]) * second_vecs[1] + (x|second_vecs[0]) * first_vecs[0])
    )

outermorphism(rotation, G_normalized, alg), G_normalized, outermorphism(rotation, G, alg), G

(-0.682 𝐞₁₂ + 0.0487 𝐞₁₃ + 0.25 𝐞₁₄ + -0.527 𝐞₁₅ + -0.356 𝐞₂₃ + -0.55 𝐞₂₄ + -0.323 𝐞₂₅ + 0.379 𝐞₃₄ + -0.694 𝐞₃₅ + -0.184 𝐞₄₅,
 -0.682 𝐞₁₂ + 0.0487 𝐞₁₃ + 0.25 𝐞₁₄ + -0.527 𝐞₁₅ + -0.356 𝐞₂₃ + -0.55 𝐞₂₄ + -0.323 𝐞₂₅ + 0.379 𝐞₃₄ + -0.694 𝐞₃₅ + -0.184 𝐞₄₅,
 -0.349 𝐞₁₂ + 0.086 𝐞₁₃ + 0.118 𝐞₁₄ + -0.156 𝐞₁₅ + -0.129 𝐞₂₃ + -0.291 𝐞₂₄ + -0.0631 𝐞₂₅ + 0.137 𝐞₃₄ + -0.156 𝐞₃₅ + -0.017 𝐞₄₅,
 -0.349 𝐞₁₂ + 0.086 𝐞₁₃ + 0.118 𝐞₁₄ + -0.156 𝐞₁₅ + -0.129 𝐞₂₃ + -0.291 𝐞₂₄ + -0.0631 𝐞₂₅ + 0.137 𝐞₃₄ + -0.156 𝐞₃₅ + -0.017 𝐞₄₅)

In [25]:
#7.26c
outermorphism(lambda x: rotation(translation(x)), G, alg), G

(-0.349 𝐞₁₂ + 0.086 𝐞₁₃ + 0.118 𝐞₁₄ + -0.156 𝐞₁₅ + -0.129 𝐞₂₃ + -0.291 𝐞₂₄ + -0.0631 𝐞₂₅ + 0.137 𝐞₃₄ + -0.156 𝐞₃₅ + -0.017 𝐞₄₅,
 -0.349 𝐞₁₂ + 0.086 𝐞₁₃ + 0.118 𝐞₁₄ + -0.156 𝐞₁₅ + -0.129 𝐞₂₃ + -0.291 𝐞₂₄ + -0.0631 𝐞₂₅ + 0.137 𝐞₃₄ + -0.156 𝐞₃₅ + -0.017 𝐞₄₅)

In [26]:
#7.28
gp = sym_part(g, alg)
matrix = trans2matrix(gp, alg)
eigvalues, eigvecs = np.linalg.eigh(matrix)
eigvecs = [alg.vector(v) for v in eigvecs.transpose()]
neg_vecs, pos_vecs, null_vecs = [], [], []
for e, v in zip(eigvalues, eigvecs):
    if e < -1e-6:
        neg_vecs.append(v)
    elif e > 1e-6:
        pos_vecs.append(v)
    else:
        null_vecs.append(v)

neg_blade = wedge(neg_vecs)
nv_sign = (-1) ** len(neg_vecs) 
c = lambda x: nv_sign*neg_blade.sw(x)
h = lambda x: sum(np.sqrt(abs(e)) * P(x, v) for e, v in zip(eigvalues, eigvecs))

x | gp(b), h(x)|c(h(b)), c(x)|h(h(b))

(3.54,
 3.54,
 3.54 + 6.39e-17 𝐞₁₂₃₄ + -6.58e-17 𝐞₁₂₃₅ + 5.65e-17 𝐞₁₂₄₅ + -6.88e-17 𝐞₁₃₄₅ + 1.08e-16 𝐞₂₃₄₅)

In [27]:
#7.28 If the symmetric transformation is positive definite (Euclidean metric)
gpp = lambda x: sum(abs(e) * P(x, v) for e, v in zip(eigvalues, eigvecs))
hinv = lambda x: sum(1/np.sqrt(abs(e)) * P(x, v) for e, v in zip(eigvalues, eigvecs))
r = 3 # a arbitrary integer <= n
Q = random_versor(r, alg)
Q = Q / norm(Q)
f = lambda x: -hinv(Q.sw(h(x)))
x | gpp(b), f(x) | gpp(f(b)), x, f(x)

(3.48,
 3.48 + -9.68e-34 𝐞₁₂ + -4.2e-33 𝐞₁₃ + 9.05e-34 𝐞₁₄ + -4.23e-34 𝐞₁₅ + 1.44e-33 𝐞₂₃ + -1.7e-33 𝐞₂₄ + -1.24e-33 𝐞₂₅ + -1.89e-33 𝐞₃₄ + -3.68e-33 𝐞₃₅ + 1.48e-33 𝐞₄₅,
 0.584 𝐞₁ + 0.763 𝐞₂ + 0.39 𝐞₃ + 0.121 𝐞₄ + 0.742 𝐞₅,
 -1.5 𝐞₁ + 0.508 𝐞₂ + 2.92 𝐞₃ + 1.21 𝐞₄ + -2.04 𝐞₅ + -4.58e-34 𝐞₁₂₃ + 1.05e-33 𝐞₁₂₅ + -5.23e-33 𝐞₁₃₄ + 2.03e-33 𝐞₁₃₅ + -4.95e-34 𝐞₁₄₅ + -4.62e-33 𝐞₂₃₄ + -4.06e-33 𝐞₂₃₅ + 2.4e-33 𝐞₂₄₅ + 2.01e-33 𝐞₃₄₅)

In [28]:
# But if gp is not positive definite, the orthogonal transform is not an isometry
f = lambda x: hinv(Q.sw(h(x)))
x | gp(b), f(x) | gp(f(b)), f(x)

(3.54,
 2.73 + 5.29e-33 𝐞₁₂ + 1.3e-33 𝐞₁₃ + -2.3e-33 𝐞₁₄ + 2.23e-33 𝐞₁₅ + -2.34e-33 𝐞₂₃ + -9.18e-33 𝐞₂₄ + -6.52e-33 𝐞₂₅ + -6.61e-33 𝐞₃₄ + -6.25e-33 𝐞₃₅ + 1.14e-32 𝐞₄₅,
 1.5 𝐞₁ + -0.508 𝐞₂ + -2.92 𝐞₃ + -1.21 𝐞₄ + 2.04 𝐞₅ + 4.58e-34 𝐞₁₂₃ + -1.05e-33 𝐞₁₂₅ + 5.23e-33 𝐞₁₃₄ + -2.03e-33 𝐞₁₃₅ + 4.95e-34 𝐞₁₄₅ + 4.62e-33 𝐞₂₃₄ + 4.06e-33 𝐞₂₃₅ + -2.4e-33 𝐞₂₄₅ + -2.01e-33 𝐞₃₄₅)

In [29]:
#7.33 isometry of c
def ortho_split(x, A):
    xp = P(x, A)
    return xp, x-xp

xp, xo = ortho_split(x, neg_blade)
bp, bo = ortho_split(b, neg_blade)
x | c(b), (xo | bo) - (xp | bp)


(1.61 + 6.18e-17 𝐞₁₂₃₄ + -1.01e-17 𝐞₁₂₃₅ + 3.25e-17 𝐞₁₂₄₅ + -6.36e-17 𝐞₁₃₄₅ + 4.86e-17 𝐞₂₃₄₅,
 1.61)

In [30]:
# Let R be a rotor (see next section) preserves the reflected subspace
R = 1
if len(neg_vecs) >= 2:
    R *= blade_exp(3.3 * wedge(neg_vecs[:2]))
if len(pos_vecs) >= 2:
    R *= blade_exp(5.5 * wedge(pos_vecs[:2]))
R.sw(neg_blade), neg_blade

(-0.264 𝐞₁₂ + 0.412 𝐞₁₃ + 0.0902 𝐞₁₄ + -0.166 𝐞₁₅ + -0.39 𝐞₂₃ + 0.146 𝐞₂₄ + -0.232 𝐞₂₅ + -0.36 𝐞₃₄ + 0.606 𝐞₃₅ + -0.0123 𝐞₄₅,
 -0.264 𝐞₁₂ + 0.412 𝐞₁₃ + 0.0902 𝐞₁₄ + -0.166 𝐞₁₅ + -0.39 𝐞₂₃ + 0.146 𝐞₂₄ + -0.232 𝐞₂₅ + -0.36 𝐞₃₄ + 0.606 𝐞₃₅ + -0.0123 𝐞₄₅)

In [31]:
# an isometry
f = lambda x: hinv(R.sw(h(x)))
x | gp(b), f(x) | gp(f(b)), f(x)

(3.54,
 3.54 + 8.91e-33 𝐞₁₂ + 7.34e-33 𝐞₁₃ + -2.71e-33 𝐞₁₄ + -1.12e-32 𝐞₁₅ + 5.24e-33 𝐞₂₃ + 4.44e-33 𝐞₂₄ + 3.8e-33 𝐞₂₅ + 6.39e-33 𝐞₃₄ + 8.16e-33 𝐞₃₅ + 4.27e-33 𝐞₄₅,
 0.309 𝐞₁ + 0.653 𝐞₂ + 0.845 𝐞₃ + 0.428 𝐞₄ + 0.58 𝐞₅ + 2.26e-33 𝐞₁₂₃ + 4.42e-33 𝐞₁₂₄ + 1.13e-33 𝐞₁₂₅ + 5.23e-33 𝐞₁₃₄ + 2.36e-33 𝐞₁₃₅ + 5.24e-33 𝐞₁₄₅ + 1.31e-33 𝐞₂₃₄ + -2.29e-34 𝐞₂₃₅ + -1.05e-33 𝐞₂₄₅ + -3.12e-33 𝐞₃₄₅)

In [32]:
# From the next section O(p, q) is easily a versor form Algebra(p, q)
# If we choose a versor from eigenvectors
R = wedge([eigvecs[i] for i in (0,2,3)]) # eigenvectors are orthogonal
R = (R / norm(R))
f = lambda x: hinv(R.sw(h(x)))
x | gp(b), f(x) | gp(f(b)), f(x)

(3.54,
 3.54 + -1.88e-33 𝐞₁₂ + -8.13e-34 𝐞₁₃ + 1.05e-33 𝐞₁₄ + -1.05e-33 𝐞₁₅ + -6.43e-34 𝐞₂₃ + -1.93e-33 𝐞₂₄ + 4.08e-35 𝐞₂₅ + -1.24e-33 𝐞₃₄ + 4.84e-34 𝐞₃₅ + 1.05e-33 𝐞₄₅,
 -0.874 𝐞₁ + -0.287 𝐞₂ + -0.0398 𝐞₃ + -0.84 𝐞₄ + -0.295 𝐞₅)

In [33]:
# From the next section O(p, q) is easily a versor form Algebra(p, q)
# If we choose a versor from eigenvectors
B = 1j * (pos_vecs[0] ^ neg_vecs[1])
R = blade_exp(B)
f = lambda x: hinv(R.sw(h(x)))
x | gp(b), f(x) | gp(f(b)), f(x)

(3.54,
 (3.6759652115448227+0.142829673511243j),
 (0.5965026633835022-0.1476224056698553j) 𝐞₁ + (1.1062016091733535-0.10156749881168206j) 𝐞₂ + (-0.013357004694264651+0.5859444564716221j) 𝐞₃ + (0.11046774510800544+0.09181640637868152j) 𝐞₄ + (0.7306611414745235-0.2352078904943536j) 𝐞₅)

In [34]:
signature = (len(pos_vecs), len(neg_vecs))
pseudo = Algebra(*signature)
O = random_versor(3, pseudo)
O = O / norm(O)

In [35]:
blade_names = pseudo.blades.blades.keys()
def parse_e_string(e_string):
    # Extract the numeric part of the string and convert each character to an integer
    return tuple(int(char) - 1 for char in e_string[1:])

[parse_e_string(k) for k in blade_names]

[(),
 (0,),
 (1,),
 (2,),
 (3,),
 (4,),
 (0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (1, 2),
 (1, 3),
 (1, 4),
 (2, 3),
 (2, 4),
 (3, 4),
 (0, 1, 2),
 (0, 1, 3),
 (0, 1, 4),
 (0, 2, 3),
 (0, 2, 4),
 (0, 3, 4),
 (1, 2, 3),
 (1, 2, 4),
 (1, 3, 4),
 (2, 3, 4),
 (0, 1, 2, 3),
 (0, 1, 2, 4),
 (0, 1, 3, 4),
 (0, 2, 3, 4),
 (1, 2, 3, 4),
 (0, 1, 2, 3, 4)]

In [36]:
R = 0
bases = pos_vecs + neg_vecs
for key in blade_names:
    indices = parse_e_string(key)
    mv = wedge([bases[i] for i in indices])
    R += O[key] * mv
R = R / norm(R)
f = lambda x: hinv(R.sw(h(x)))
x | gp(b), f(x) | gp(f(b)), f(x)

(3.54,
 -0.495 + 2.11e-32 𝐞₁₂ + -1.5e-32 𝐞₁₃ + 1.34e-32 𝐞₁₄ + 5.43e-33 𝐞₁₅ + -1.31e-32 𝐞₂₃ + -2.36e-32 𝐞₂₄ + 3.3e-33 𝐞₂₅ + 6.41e-33 𝐞₃₄ + -1.72e-32 𝐞₃₅ + -5.91e-33 𝐞₄₅,
 1.71 𝐞₁ + -0.699 𝐞₂ + 0.626 𝐞₃ + -0.043 𝐞₄ + -1.64 𝐞₅ + 3.06e-33 𝐞₁₂₃ + -2.09e-32 𝐞₁₂₄ + 2.62e-33 𝐞₁₂₅ + 6.28e-33 𝐞₁₃₄ + -9.24e-33 𝐞₁₃₅ + 2.01e-33 𝐞₁₄₅ + 1.01e-32 𝐞₂₃₄ + -8.42e-33 𝐞₂₃₅ + -4.21e-33 𝐞₂₄₅ + -6.06e-33 𝐞₃₄₅)

So a rotation preserves the reflected subspace is an isometry. Any other isometry? The above we split the space into pos_vecs and neg_vecs. The metric's signature encode such split. The next section we encode the metric into the algebra. So an orthogonal transform becomes an isometry of the vector space.

In [37]:
# signature
[v | c(v) for v in eigvecs]

[-1.0 + 9.01e-18 𝐞₁₂₃₄ + 5.43e-18 𝐞₁₂₃₅ + 3.46e-18 𝐞₁₂₄₅ + -4.47e-18 𝐞₁₃₄₅ + -7.07e-18 𝐞₂₃₄₅,
 -1.0,
 1.0 + 1.17e-17 𝐞₁₂₃₄ + 3.69e-18 𝐞₁₂₃₅ + -4.15e-18 𝐞₁₂₄₅ + 1.51e-17 𝐞₁₃₄₅ + 6.19e-18 𝐞₂₃₄₅,
 1.0 + 6.45e-18 𝐞₁₂₃₄ + -2.19e-17 𝐞₁₂₃₅ + -5.22e-18 𝐞₁₂₄₅ + 4.71e-18 𝐞₁₃₄₅ + -1.41e-17 𝐞₂₃₄₅,
 1.0]

In [38]:
p, q, r = 0,0,0
for e in eigvalues:
    if e > 1e-6:
        p += 1
    elif e < -1e-6:
        q += 1
    else:
        r += 1
p, q, r

(3, 2, 0)

In [39]:
#8.2 Say the reflected subspace is 1d
alg = Algebra(3, 1)
locals().update(alg.blades)
x, y, u = random_r_vectors(3, alg)
u = u / norm(u)
R = lambda x: - u.sw(x)
R(x) | R(y), x | y

(0.8, 0.8)

Cartan-Dieudonne theorem: An isometry is a composition of at most d reflections. Identifying invariants of isometries as elements of geometry leads to modern developments.

In [40]:
#8.3 Cartan-Dieudonne theorem: An isometry is a composition of at most d reflections.

k = 3 # k <= d
vectors = random_r_vectors(k, alg)
p = sum((v**2)[0] > 0 for v in vectors)
U = gprod(vectors) # a versor, of either even or odd grades
f = lambda x: (-1) ** k * U * x / U
f_ = lambda x: (-1) ** p * U.sw(x) / normsq(U) #8.8b
f(x) | f(y), x | y, f_(x) | f_(y)

(0.8 + -2.38e-17 𝐞₁₂ + 7.55e-17 𝐞₁₃ + -6.38e-17 𝐞₁₄ + 4.69e-17 𝐞₂₃ + 2.79e-17 𝐞₂₄ + 6.64e-18 𝐞₃₄,
 0.8,
 0.8)

In [41]:
#8.8
def random_versor(r, alg):
    return gprod(random_r_vectors(r, alg))

def random_versor_(r, alg):
    vectors = random_r_vectors(r, alg)
    p = sum((v**2)[0] > 0 for v in vectors)
    return gprod(vectors), p
k = 3
for _ in range(10):
    U, p = random_versor_(k, alg)
    U = U/norm(U)
    print(k - p, U.reverse() * U)

1 -1.0 + 3.55e-15 𝐞₁₃ + -7.55e-15 𝐞₁₄ + -1.78e-15 𝐞₃₄ + -3.55e-15 𝐞₁₂₃₄
0 1.0 + -2.78e-17 𝐞₁₃ + -9.71e-17 𝐞₁₄ + -5.55e-17 𝐞₂₃ + -3.47e-17 𝐞₂₄ + -5.55e-17 𝐞₃₄ + -1.11e-16 𝐞₁₂₃₄
1 -1.0 + 1.11e-16 𝐞₁₃ + -3.33e-16 𝐞₁₄ + -4.16e-17 𝐞₂₃ + 8.88e-16 𝐞₂₄ + 1.11e-16 𝐞₃₄ + 2.22e-16 𝐞₁₂₃₄
0 1.0 + -2.43e-17 𝐞₁₃ + 6.94e-18 𝐞₁₄ + 8.33e-17 𝐞₂₃ + -2.78e-17 𝐞₃₄ + -8.33e-17 𝐞₁₂₃₄
0 1.0 + -2.78e-17 𝐞₁₂ + 4.34e-18 𝐞₁₃ + -1.11e-16 𝐞₁₄ + -5.2e-18 𝐞₂₃ + 6.61e-18 𝐞₃₄ + 2.36e-16 𝐞₁₂₃₄
0 1.0 + 5.2e-18 𝐞₁₂ + -1.11e-16 𝐞₂₄ + 1.67e-16 𝐞₁₂₃₄
0 1.0 + 4.44e-16 𝐞₃₄ + 1.78e-15 𝐞₁₂₃₄
0 1.0 + 4.86e-17 𝐞₁₃ + -3.82e-17 𝐞₁₄ + -4.16e-17 𝐞₂₃ + -1.39e-17 𝐞₂₄ + 1.39e-17 𝐞₃₄ + -1.46e-16 𝐞₁₂₃₄
0 1.0 + -3.47e-18 𝐞₁₂ + -1.3e-17 𝐞₁₃ + -1.56e-17 𝐞₁₄ + 8.67e-19 𝐞₂₃ + -4.99e-18 𝐞₂₄ + 3.04e-18 𝐞₁₂₃₄
1 -1.0 + -1.25e-15 𝐞₁₃ + -6.01e-15 𝐞₁₄ + -2.66e-15 𝐞₂₄ + 1.55e-15 𝐞₃₄ + 1.47e-14 𝐞₁₂₃₄


In [42]:
# versors (Pin) are 2:1 homomorphisc to orthogonal transformations (O)
U = random_versor(3, alg)
U.sw(x), (-U).sw(x)

(0.00737 𝐞₁ + -0.00487 𝐞₂ + -0.00378 𝐞₃ + -0.00156 𝐞₄,
 0.00737 𝐞₁ + -0.00487 𝐞₂ + -0.00378 𝐞₃ + -0.00156 𝐞₄)

In [43]:
#8.5
I = alg.pseudoscalar((1,))
I.sw(x), x

(0.41 𝐞₁ + 0.159 𝐞₂ + 0.669 𝐞₃ + 0.0017 𝐞₄,
 0.41 𝐞₁ + 0.159 𝐞₂ + 0.669 𝐞₃ + 0.0017 𝐞₄)

In [44]:
#8.7 orthogonal transformations (O) with determinant 1 (SO) correspond to even versors (Spin)
for k in range(1,5):
    U = gprod(random_r_vectors(k, alg)) # a versor, of either even or odd grades
    f = lambda x: (-1) ** (k*alg.d) * U.sw(x)/normsq(U)
    print(k, outermorphism(f, I, alg))

1 -1.0 𝐞₁₂₃₄
2 1.0 𝐞₁₂₃₄
3 -1.0 𝐞₁₂₃₄
4 1.0 𝐞₁₂₃₄


In [45]:
#8.9 rotors (Spin+) are a part of even versors (Spin), correpond to a part of SO as rotations (SO+)
def random_even_versor(alg):
    d = alg.d
    if (alg.d % 2):
        d -= 1
    return gprod(random_r_vectors(d, alg))


def random_rotor(alg, max_iter=100):
    for _ in range(max_iter):
        S = random_even_versor(alg)
        sign  = (S.reverse().sp(S))[0]
        if sign > 0:
            return S/norm(S)
        

def random_rotor_count(alg, max_iter=100):
    count = 0
    for _ in range(max_iter):
        S = random_even_versor(alg)
        sign  = (S.reverse().sp(S))[0]
        if sign > 0:
            count += 1
    return count

[(p, q, r, random_rotor_count(Algebra(p, q, r), 2000)/2000) for p, q, r in ((3,1,0),(1,3,0),(4,1,0),(3,0,1),(2,2,0))]

[(3, 1, 0, 0.6415),
 (1, 3, 0, 0.672),
 (4, 1, 0, 0.8055),
 (3, 0, 1, 1.0),
 (2, 2, 0, 0.492)]

versors (Pin) are products of {1, a, b, ab} and a rotor

In [46]:
#8.12
def random_simple_rotor(alg):
    while True:
        S = random_versor(2, alg)
        sign  = (S.reverse().sp(S))[0]
        if sign > 0:
            return S/norm(S)

In [47]:
#8.13
S = random_simple_rotor(alg)
u = x | S.grade(2)
u = u/norm(u)
v = S.sw(u)
v, S ** 2 * u, v ** 2, u ** 2, v * u, S ** 2

(0.598 𝐞₁ + 0.395 𝐞₂ + 1.06 𝐞₃ + 0.798 𝐞₄,
 0.598 𝐞₁ + 0.395 𝐞₂ + 1.06 𝐞₃ + 0.798 𝐞₄ + 5.75e-17 𝐞₁₂₃ + 1.4e-16 𝐞₁₂₄ + 3.26e-17 𝐞₁₃₄ + 2.6e-18 𝐞₂₃₄,
 1.0,
 1.0,
 0.02 + -0.119 𝐞₁₂ + -1.15 𝐞₁₃ + -0.63 𝐞₁₄ + -0.549 𝐞₂₃ + -0.258 𝐞₂₄ + 0.418 𝐞₃₄,
 0.02 + -0.119 𝐞₁₂ + -1.15 𝐞₁₃ + -0.63 𝐞₁₄ + -0.549 𝐞₂₃ + -0.258 𝐞₂₄ + 0.418 𝐞₃₄ + 1.04e-16 𝐞₁₂₃₄)

In [48]:
S ** 2 * (S ** 2).reverse(), S * S.reverse()

(1.0 + -2.78e-17 𝐞₁₂ + -2.68e-17 𝐞₁₃ + -1.61e-18 𝐞₁₄ + -3.78e-18 𝐞₂₃ + 8.68e-18 𝐞₂₄ + -1.93e-17 𝐞₃₄ + -1.99e-16 𝐞₁₂₃₄,
 1.0 + 5.55e-17 𝐞₁₃ + -2.78e-17 𝐞₁₄ + 1.39e-17 𝐞₂₄ + -1.04e-16 𝐞₁₂₃₄)

In [49]:
#8.15
(S ** 2).reverse(), u * v, v * u

(0.02 + 0.119 𝐞₁₂ + 1.15 𝐞₁₃ + 0.63 𝐞₁₄ + 0.549 𝐞₂₃ + 0.258 𝐞₂₄ + -0.418 𝐞₃₄ + 1.04e-16 𝐞₁₂₃₄,
 0.02 + 0.119 𝐞₁₂ + 1.15 𝐞₁₃ + 0.63 𝐞₁₄ + 0.549 𝐞₂₃ + 0.258 𝐞₂₄ + -0.418 𝐞₃₄,
 0.02 + -0.119 𝐞₁₂ + -1.15 𝐞₁₃ + -0.63 𝐞₁₄ + -0.549 𝐞₂₃ + -0.258 𝐞₂₄ + 0.418 𝐞₃₄)

In [50]:
#8.16
S2 = S ** 2
blade = S2.grade(2)
bladesq = (blade ** 2)
bladesq

-1.0 + 2.78e-17 𝐞₁₂ + 2.01e-16 𝐞₁₂₃₄

In [51]:
# 3 types of rotors (Spin+)
def type_convert(n):
    if n > 1e-6:
        return 1
    elif n < -1e-6:
        return -1
    else:
        return 0

# Unfortunately parabolic rotors are of measure zero, 
# so this function has zero chance generating them
def random_simple_rotor(alg, signature=None, max_iter=100):
    for _ in range(max_iter):
        S = random_versor(2, alg)
        if signature is not None:
            if type_convert((S.grade(2) ** 2)[0]) != signature:
                continue
        sign  = (S.reverse().sp(S))[0]
        if sign > 0:
            return S/norm(S)
        
[random_simple_rotor(alg, signature=i) for i in range(-1, 2)]

[0.842 + 0.147 𝐞₁₂ + 0.18 𝐞₁₃ + 0.119 𝐞₁₄ + 0.623 𝐞₂₃ + 0.128 𝐞₂₄ + -0.347 𝐞₃₄,
 None,
 1.17 + 0.999 𝐞₁₂ + -0.298 𝐞₁₃ + 1.07 𝐞₁₄ + -1.04 𝐞₂₃ + -0.164 𝐞₂₄ + 1.17 𝐞₃₄]

In [52]:
def simple_rotor_log(R: MultiVector, tol=1e-6):
    blade = R.grade(2)
    signature = (blade ** 2)[0]
    if signature > tol:
        return np.arccosh(R[0]) / norm(blade) * blade
    if signature < -tol:
        return np.arccos(R[0]) / norm(blade) * blade
    else:
        return blade

In [53]:
#8.17a elliptic rotor (rotation), run several times you can see sqrt allows a sign change
S = random_simple_rotor(alg, signature=-1)
S2 = S ** 2
b = simple_rotor_log(S2)
S2, blade_exp(b), blade_exp(b - 6*np.pi*b/norm(b)), S, blade_exp(b/2), simple_rotor_sqrt(S2), blade_exp(3.3*b).sw(S)

(0.385 + 0.604 𝐞₁₂ + -0.613 𝐞₁₃ + -0.134 𝐞₁₄ + -0.582 𝐞₂₃ + -0.38 𝐞₂₄ + 0.256 𝐞₃₄ + -3.47e-17 𝐞₁₂₃₄,
 0.385 + 0.604 𝐞₁₂ + -0.613 𝐞₁₃ + -0.134 𝐞₁₄ + -0.582 𝐞₂₃ + -0.38 𝐞₂₄ + 0.256 𝐞₃₄,
 0.385 + 0.604 𝐞₁₂ + -0.613 𝐞₁₃ + -0.134 𝐞₁₄ + -0.582 𝐞₂₃ + -0.38 𝐞₂₄ + 0.256 𝐞₃₄,
 0.832 + 0.363 𝐞₁₂ + -0.368 𝐞₁₃ + -0.0806 𝐞₁₄ + -0.35 𝐞₂₃ + -0.228 𝐞₂₄ + 0.154 𝐞₃₄,
 0.832 + 0.363 𝐞₁₂ + -0.368 𝐞₁₃ + -0.0806 𝐞₁₄ + -0.35 𝐞₂₃ + -0.228 𝐞₂₄ + 0.154 𝐞₃₄,
 0.832 + 0.363 𝐞₁₂ + -0.368 𝐞₁₃ + -0.0806 𝐞₁₄ + -0.35 𝐞₂₃ + -0.228 𝐞₂₄ + 0.154 𝐞₃₄ + -2.08e-17 𝐞₁₂₃₄,
 0.832 + 0.363 𝐞₁₂ + -0.368 𝐞₁₃ + -0.0806 𝐞₁₄ + -0.35 𝐞₂₃ + -0.228 𝐞₂₄ + 0.154 𝐞₃₄ + 1.25e-16 𝐞₁₂₃₄)

In [54]:
#8.17b hyperbolic rotor (boost)
S = random_simple_rotor(alg, signature=1)
S2 = S ** 2
b = simple_rotor_log(S2)
S2, blade_exp(b), S, blade_exp(b/2), simple_rotor_sqrt(S2), blade_exp(3.3*b).sw(S)

(1.12 + -0.181 𝐞₁₂ + -0.0842 𝐞₁₃ + -0.275 𝐞₁₄ + 0.399 𝐞₂₃ + -0.0216 𝐞₂₄ + -0.616 𝐞₃₄ + -6.94e-18 𝐞₁₂₃₄,
 1.12 + -0.181 𝐞₁₂ + -0.0842 𝐞₁₃ + -0.275 𝐞₁₄ + 0.399 𝐞₂₃ + -0.0216 𝐞₂₄ + -0.616 𝐞₃₄,
 1.03 + -0.088 𝐞₁₂ + -0.0409 𝐞₁₃ + -0.133 𝐞₁₄ + 0.194 𝐞₂₃ + -0.0105 𝐞₂₄ + -0.299 𝐞₃₄,
 1.03 + -0.088 𝐞₁₂ + -0.0409 𝐞₁₃ + -0.133 𝐞₁₄ + 0.194 𝐞₂₃ + -0.0105 𝐞₂₄ + -0.299 𝐞₃₄,
 1.03 + -0.088 𝐞₁₂ + -0.0409 𝐞₁₃ + -0.133 𝐞₁₄ + 0.194 𝐞₂₃ + -0.0105 𝐞₂₄ + -0.299 𝐞₃₄ + -3.37e-18 𝐞₁₂₃₄,
 1.03 + -0.088 𝐞₁₂ + -0.0409 𝐞₁₃ + -0.133 𝐞₁₄ + 0.194 𝐞₂₃ + -0.0105 𝐞₂₄ + -0.299 𝐞₃₄ + 8.88e-16 𝐞₁₂₃₄)

In [55]:
#8.17c 8.20 parabolic rotor (translation)
S = (e2 + e3 - e4) * e2
S2 = S ** 2
b = simple_rotor_log(S2)
S2, blade_exp(b), S, blade_exp(b/2), simple_rotor_sqrt(S2), blade_exp(3.3*b).sw(S)

(1 + -2 𝐞₂₃ + 2 𝐞₂₄,
 1 + -2 𝐞₂₃ + 2 𝐞₂₄,
 1 + -1 𝐞₂₃ + 1 𝐞₂₄,
 1 + -1.0 𝐞₂₃ + 1.0 𝐞₂₄,
 1.0 + -1.0 𝐞₂₃ + 1.0 𝐞₂₄,
 1.0 + -1.0 𝐞₂₃ + 1.0 𝐞₂₄)

In [56]:
#8.21
B = S.grade(2) / (1 + S.grade(0))
B, (1+B)**2/(normsq(1+B)), S

(-0.5 𝐞₂₃ + 0.5 𝐞₂₄, 1.0 + -1.0 𝐞₂₃ + 1.0 𝐞₂₄, 1 + -1 𝐞₂₃ + 1 𝐞₂₄)

In [57]:
#8.25
R, u = random_rotor(alg), random_r_blade(1, alg)
v = R.sw(u)
S = simple_rotor_sqrt(v * u / normsq(u))
U = S.reverse() * R
R, S * U, S * u, u * S.reverse(), U * u, u * U

(-2.05 + 2.84 𝐞₁₂ + 0.989 𝐞₁₃ + 3.48 𝐞₁₄ + 1.15 𝐞₂₃ + 0.525 𝐞₂₄ + -1.05 𝐞₃₄ + -0.229 𝐞₁₂₃₄,
 -2.05 + 2.84 𝐞₁₂ + 0.989 𝐞₁₃ + 3.48 𝐞₁₄ + 1.15 𝐞₂₃ + 0.525 𝐞₂₄ + -1.05 𝐞₃₄ + -0.229 𝐞₁₂₃₄,
 1.61 𝐞₁ + 2.48 𝐞₂ + 0.92 𝐞₃ + 2.95 𝐞₄ + -2.22e-16 𝐞₁₂₃ + -2.78e-17 𝐞₁₂₄ + -2.22e-16 𝐞₁₃₄ + -2.78e-17 𝐞₂₃₄,
 1.61 𝐞₁ + 2.48 𝐞₂ + 0.92 𝐞₃ + 2.95 𝐞₄ + 2.22e-16 𝐞₁₂₃ + 2.22e-16 𝐞₁₃₄ + 2.78e-17 𝐞₂₃₄,
 -0.96 𝐞₁ + -0.0205 𝐞₂ + -0.573 𝐞₃ + -0.131 𝐞₄ + 1.32 𝐞₁₂₃ + 0.394 𝐞₁₂₄ + -1.4 𝐞₁₃₄ + -0.0839 𝐞₂₃₄,
 -0.96 𝐞₁ + -0.0205 𝐞₂ + -0.573 𝐞₃ + -0.131 𝐞₄ + 1.32 𝐞₁₂₃ + 0.394 𝐞₁₂₄ + -1.4 𝐞₁₃₄ + -0.0839 𝐞₂₃₄)

In [58]:
def rotor_split(R: MultiVector, alg: Algebra, tol=1e-6):
    # FIXME: when the tangent not well defined
    assert normsq(R.grade(2)) > tol, "zero bivector"
    tangent = R.grade(2) / R[0]
    tangents = bivector_split(tangent, alg)
    splits = sorted([normalize(1 + t) for t in tangents], key=lambda item: abs(item[0]))
    sign = np.round(gprod(r[0] for r in splits)/R[0])
    splits[0] = splits[0] * sign # to keep the sign of R
    return splits
        

R_splits = rotor_split(R, alg)
R, gprod(R_splits), gprod([blade_exp(simple_rotor_log(r)) for r in R_splits])

(-2.05 + 2.84 𝐞₁₂ + 0.989 𝐞₁₃ + 3.48 𝐞₁₄ + 1.15 𝐞₂₃ + 0.525 𝐞₂₄ + -1.05 𝐞₃₄ + -0.229 𝐞₁₂₃₄,
 -2.05 + 2.84 𝐞₁₂ + 0.989 𝐞₁₃ + 3.48 𝐞₁₄ + 1.15 𝐞₂₃ + 0.525 𝐞₂₄ + -1.05 𝐞₃₄ + -0.229 𝐞₁₂₃₄,
 -2.05 + 2.84 𝐞₁₂ + 0.989 𝐞₁₃ + 3.48 𝐞₁₄ + 1.15 𝐞₂₃ + 0.525 𝐞₂₄ + -1.05 𝐞₃₄ + -0.229 𝐞₁₂₃₄)

In [59]:
#8.10 a spinor is an even versor for odd dimension (a Dirac representation);
# a sum of an even versor and its dual for even dimension (two Weyl representations)
def random_study_number(alg):
    a, b = np.random.rand(2)
    return (a + b*alg.pseudoscalar((1,)))

def random_spinor(alg: Algebra):
    if alg.d % 4:
        return random_even_versor(alg)
    return random_study_number(alg) * random_even_versor(alg)

random_spinor(Algebra(3)), random_spinor(Algebra(4))


(0.0847 + -0.0432 𝐞₁₂ + 0.00199 𝐞₁₃ + 0.107 𝐞₂₃,
 0.713 + 0.111 𝐞₁₂ + 0.124 𝐞₁₃ + 0.25 𝐞₁₄ + -0.101 𝐞₂₃ + 0.295 𝐞₂₄ + 0.161 𝐞₃₄ + 0.133 𝐞₁₂₃₄)

In [60]:
# A pointor depends on a point
def random_point(alg: Algebra):
    return random_r_blade(alg.d - 1, alg)

O = random_point(alg)

# point reflection on the point itself is a pointor
terms_ratio(O, O.sw(O))

array([-33.57429333, -33.57429333, -33.57429333, -33.57429333])

In [61]:
# versors not crossing O-subspace are pointers 
vectors = blade_split(O, alg)
B1, B2 = wedge(vectors[:2]), wedge(vectors[1:])
Od = O.dual()
terms_ratio(O, Od.sw(O)), terms_ratio(B1.sw(O), O), terms_ratio(vectors[0].sw(O), O), 

(array([-33.57429333, -33.57429333, -33.57429333, -33.57429333]),
 array([12.90591293, 12.90591293, 12.90591293, 12.90591293]),
 array([13.36279296, 13.36279296, 13.36279296, 13.36279296]))

In [62]:
# rotors with reflections are pointers
R1 = blade_exp(3.3*B1)
R2 = blade_exp(4.4*B2)
terms_ratio((Od * R1 * R2).sw(O), O), terms_ratio((Od * vectors[2] * R1).sw(O), O)

(array([-0.02978469, -0.02978469, -0.02978469, -0.02978469]),
 array([0.03264996, 0.03264996, 0.03264996, 0.03264996]))

In [63]:
# rotors cross O-subspace may not be pointers
terms_ratio(blade_exp(Od ^ vectors[2]).sw(O), O)

array([2.31265738, 1.17541562, 1.17541562, 1.17541562])

In [64]:
# A general pointor suggested by "From Invariant Decomposition to Spinors"?
# How do we find the point specified by a pointor?
def random_odd_versor(alg):
    d = alg.d
    if not (alg.d % 2):
        d -= 1
    return gprod(random_r_vectors(d, alg))


# Actually I'm not sure these generate pointors
# def random_pointor(alg: Algebra):
#     return random_odd_versor(alg) * random_even_versor(alg)
    
# def random_pointor(alg: Algebra):
#     v = random_r_blade(1, alg)
#     return (np.random.rand() + v) * random_even_versor(alg)

# FIXME: check degree of freedom
def random_pointor(alg: Algebra):
    v = random_r_blade(1, alg)
    return (np.random.rand() + v) * random_versor(2, alg)

pointor = random_pointor(alg)
pointor

0.485 + 0.228 𝐞₁ + 0.447 𝐞₂ + 0.646 𝐞₃ + 0.0591 𝐞₄ + -0.0795 𝐞₁₂ + -0.0623 𝐞₁₃ + -0.0573 𝐞₁₄ + 0.116 𝐞₂₃ + -0.032 𝐞₂₄ + -0.108 𝐞₃₄ + 0.00599 𝐞₁₂₃ + 0.0281 𝐞₁₂₄ + 0.0177 𝐞₁₃₄ + -0.0433 𝐞₂₃₄

In [65]:
# the point-grade
O = pointor.grade(3)
terms_ratio(pointor.sw(O), O)

array([0.90449692, 0.90449692, 0.90449692, 0.90449692])